# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

## Лабораторная работа №4

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [1]:
import json

with open('contributors_sample.json', 'r', encoding='utf-8') as file:
    contributors_data = json.load(file)

for contributor in contributors_data[:3]:
    print(contributor)

{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}
{'username': 'vickitaylor', 'name': 'Cheryl Lewis', 'sex': 'F', 'address': '66992 Welch Brooks\nMarshallshire, ID 56004', 'mail': 'bhudson@gmail.com', 'jobs': ['Music therapist', 'Volunteer coordinator', 'Designer, interior/spatial'], 'id': 91970}
{'username': 'sheilaadams', 'name': 'Julia Allen', 'sex': 'F', 'address': 'Unit 1632 Box 2971\nDPO AE 23297', 'mail': 'darren44@yahoo.com', 'jobs': ['Management consultant', 'Engineer, structural', 'Lecturer, higher education', 'Theatre manager', 'Designer, textile'], 'id': 1848091}


1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

Количество уникальных почтовых доменов: 0
Уникальные почтовые домены: set()


1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [8]:
import json

def find_user_info(username):
    with open('contributors_sample.json', 'r', encoding='utf-8') as file:
        contributors_data = json.load(file)

    for contributor in contributors_data:
        if 'username' in contributor and contributor['username'] == username:      
            print("Информация о пользователе:")
            print("Имя пользователя:", contributor['username'])
            print("Почтовый адрес:", contributor.get('email', 'Нет почтового адреса'))
            return
    raise ValueError("Пользователь с username '{}' не найден".format(username))
try:
    find_user_info('john_doe')
except ValueError as e:
    print(e)

Пользователь с username 'john_doe' не найден


1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

Количество мужчин: 0
Количество женщин: 0


1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [17]:
import pandas as pd
import json

with open('contributors_sample.json', 'r', encoding='utf-8') as file:
    contributors_data = json.load(file)

data = []

for contributor in contributors_data:
    contributor_id = contributor.get('id', None)
    username = contributor.get('username', None)
    sex = contributor.get('gender', None)
    data.append({'id': contributor_id, 'username': username, 'sex': sex})

contributors = pd.DataFrame(data)

print(contributors.head())

        id     username   sex
0    35193      uhebert  None
1    91970  vickitaylor  None
2  1848091  sheilaadams  None
3    50969     nicole82  None
4   676820       jean67  None


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [18]:
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv')

with open('contributors_sample.json', 'r', encoding='utf-8') as file:
    contributors_data = json.load(file)

contributors = pd.DataFrame(contributors_data)

merged_df = pd.merge(recipes, contributors, how='left', left_on='contributor_id', right_on='id')

missing_contributors = merged_df['username'].isna().sum()

print("Количество человек, информация о которых отсутствует в JSON-файле:", missing_contributors)

Количество человек, информация о которых отсутствует в JSON-файле: 15059


### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [20]:
import json

with open('contributors_sample.json', 'r', encoding='utf-8') as file:
    contributors_data = json.load(file)

positions_dict = {}

for contributor in contributors_data:
    position = contributor.get('position', None)
    username = contributor.get('username', None)
   
    if position:
        if position not in positions_dict:
            positions_dict[position] = []
        positions_dict[position].append(username)

for position, usernames in positions_dict.items():
    print(position, ':', usernames)

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [21]:
import pickle

with open('job_people.pickle', 'wb') as file:
    pickle.dump(positions_dict, file)


2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [24]:
import pickle

with open('job_people.pickle', 'rb') as file:
    loaded_positions_dict = pickle.load(file)

for position, usernames in loaded_positions_dict.items():
    print(position, ':', usernames)

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [25]:
import xml.etree.ElementTree as ET
import json

tree = ET.parse('steps_sample.xml')
root = tree.getroot()

steps_dict = {}

for recipe in root.findall('recipe'):
    recipe_id = recipe.get('id')
    steps = []
    for step in recipe.findall('step'):
        steps.append(step.text.strip())  
    steps_dict[recipe_id] = steps

with open('steps_sample.json', 'w', encoding='utf-8') as file:
    json.dump(steps_dict, file, ensure_ascii=False, indent=4)

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`